In [ ]:
%load_ext autoreload
%load_ext aymurai.devtools.magic
%autoreload 2

In [ ]:
import spacy
from spaczz.pipeline import SpaczzRuler

In [ ]:
examples = """
TELA s/ art. 181 inc , 1 usurpación -
art. 1472:73
 76 ter y 27 bis del CP
prevista y reprimida en el art, 72 de Código Contravencional (en " adelante, CC ).
conf. art. 4 inciso a) Ley 24.660).
 art. 7 apartado 1V inciso c) Ley 24.660.
, conforme lo previsto en el art. 7 apartado 1V inciso c) Ley 24.660.
 ART. 73 SEGÚN LEY 1472
 art. 82 inciso "a" del Código Contravencional (Ley 1472, Cfrme. Texto Ley 6347/20
 ARTÍCULOS 89 / 90 Y 91 
 art. 323 CPP. 
 Art. 149 BIS del Código Penal (en adelante, CP) 
 art. 17 del CPPCABA
 conf. arts. 7 y 17 del CPPCABA
 art. 13.3 CCABA
 arts. 59 inc. 1 del Código Penal y 211 inc "b" Código Procesal Penal).
 art. 211, inc. "b" última parte del Código Procesal Penal de la CABA (en adelante, CPP)
 art. 59, inc. 1 del Código Penal de la Nación (en adelante, CP)
 arts. 59 inc. 2, art. 211 inc "b" CPP)
 art. 213 CPPCABA
art. 172
292 1° PARR CP
art.18 de la Constitución Nacional y art. 28, inciso 2 CPPCABA
art. 232 del CPPCABA
art. 173 inc 16 del CP
art. 106 CCABA
artículo 172 del CP (arts. 5, 31 y 129 CN; 6 y 106 CCABA)
art. 18 de la Constitución Nacional y 53 de la Ley 1217
artículo 3 de la Ley 1217
art. 6.1.52 de la Ley 451
art. 7.1.9 del Código de Transporte (ley 2148)
arts. 6.1.42 y 6.1.52 de la Ley 451
arts. 19, 35, 6.1.42, 6.1.52 de la ley 451, arts 28, 48, 52 y 55 Ley 1217)
artículos 92 y 80 inc. 1 y 11 y 149 del CP; art. 5 inc. 1, 2, 4 y 5, art. 6 inc. a) Ley 26.485
Ley de Salud Mental (art. 199 inc. c) del Código Procesal Penal CABA y art. 34, inc. 1º, primer párrafo, del Código Penal, Ley 26.485
art. 199 inc. c) CPP
arts. 4 y 5.1, 5.2, 5.4 5.5 y 6 a) de la Ley 26.485 de protección integral de las mujeres
art. 120 de la Constitución Nacional y art. 244 último párrafo del CPPCABA, en consonancia con el art. 13.3 de la Constitución local, que establece que en esta Ciudad
SOBRE 129 1 PARR - 
 art. 65bis CC"
 n el art. 45, incs. 1, 2, 3, 4, 5 y 7 del Código Contravencional;
 nfracción al art. 172 CP y, por ello, de acuerdo con lo normado en las leyes 25.752, 26.357 y 26.702, su conocimiento y decisión era ajeno a esta jurisdicción.
 SOBRE 149 BIS - AMENAZA
 STIN s. art. 52 Hostigar, maltratar, intimidar - CC", del re
 FABIO SOBRE 2 BIS - INCUMPLIMIENTO DE 
 IO s. art. 52 CC",
 O SOBRE 189 BIS (2) - TE
"""

# Article patterns

In [ ]:
%%export aymurai.spacy.components.es_ar.articles.patterns
import spacy
spacy.blank('es')

ART_PREFIX = r"arts?(\.|,)?|ARTÍCULOS?|artículos?"
ART_PREFIX_TITLE = r'SOBRE'

ART_MOD_PREFIX = "incs?[\.,]?|inciso|apartado|(1[\Wo]\s+)?parr"
ART_MOD_CODE = '[\d\w"\)]+'
ART_MOD = f",?\s?(({ART_MOD_PREFIX})(\s({ART_MOD_CODE})))"


ART_CODE = r"\d[\d\.]*(\s?(bis|ter|\w\)))?"
ART_FULL = f"{ART_CODE}({ART_MOD})*"
ART_FULL_MOD = f"{ART_CODE}({ART_MOD})+"


ART_PATTERN_MULTI_NO_PREFIX = (
    r"(?i)\d+(\s?(bis|ter|\d\W\s*parr))"
)
ART_PATTERN_MULTI_PREFIX = (
    f"(?i)({ART_PREFIX})\s?({ART_FULL})(?:(,|\s(y|\/))? ({ART_FULL}))*"
)
ART_PATTERN_MULTI_PREFIX_TITLE = (
    f"(?i)({ART_PREFIX_TITLE}\s?)({ART_FULL})(?:(,|\s(y|\/))? ({ART_FULL}))*"
)
ART_PATTERN_MULTI_MOD = f"(?i)({ART_FULL_MOD})(?:(,|\s(y|\/))? ({ART_FULL_MOD}))*"


CODES = [
    "Código Contravencional",
    "Código Penal",
    "Código Procesal Penal",
    "Código Penal de la Nación",
    "Constitución Nacional",
    "Código de Transporte",
    "Código Contravencional",
    "Constitución local",
]

ABBRS = [
    "".join(
        [
            token[0]
            for token in code.split(" ")
            if token not in spacy.lang.es.STOP_WORDS and token[0].isupper()
        ]
    )
    for code in CODES
]
ABBRS = filter(lambda x: len(x) > 1, ABBRS)
ABBRS = list(ABBRS)
ABBRS


In [ ]:
nlp = spacy.blank('es')

In [ ]:
nlp = spacy.blank("es")
ruler = SpaczzRuler(nlp)
ruler.add_patterns(
    [
        {
            "label": "ARTICLE_TITLE",
            "type": "regex",
            "pattern": ART_PATTERN_MULTI_PREFIX_TITLE,
        },
        {
            "label": "ARTICLE",
            "type": "regex",
            "pattern": ART_PATTERN_MULTI_NO_PREFIX,
        },
        {
            "label": "ARTICLE",
            "type": "regex",
            "pattern": ART_PATTERN_MULTI_PREFIX,
        },
        {
            "label": "ARTICLE",
            "type": "regex",
            "pattern": ART_PATTERN_MULTI_MOD,
        },
    ]
)

doc = nlp(examples)
doc = ruler(doc)
spacy.displacy.render(doc, "ent")


# code or law patterns

In [ ]:
nlp = spacy.blank("es")
ruler = SpaczzRuler(nlp)
ruler.add_patterns(
    [
        {
            "label": "CODE_OR_LAW",
            "type": "fuzzy",
            "pattern": code,
        } for code in CODES
    ] +
    [
        {
            "label": "CODE_OR_LAW",
            "type": "regex",
            "pattern": '(?i)ley(es)?(( y|,)? ([\d\.]+))+',
        },
    ] + 
    [
        {
            "label": "CODE_OR_LAW",
            "type": "regex",
            "pattern": f'{abbr}(\s?CABA)?',
        } for abbr in ABBRS
    ]
)

doc = nlp(examples)
doc = ruler(doc)
spacy.displacy.render(doc, "ent")


In [ ]:
nlp = spacy.blank("es")
ruler = SpaczzRuler(nlp)
ruler.add_patterns(
    # [
    #     {
    #         "label": "ARTICLE",
    #         "type": "regex",
    #         "pattern": ART_PATTERN_MULTI_PREFIX,
    #     },
    #     {
    #         "label": "ARTICLE",
    #         "type": "regex",
    #         "pattern": ART_PATTERN_MULTI_MOD,
    #     },
    # ] +
    [
        {
            "label": "CODE_OR_LAW",
            "type": "fuzzy",
            "pattern": code,
        } for code in CODES
    ] +
    [
        {
            "label": "CODE_OR_LAW",
            "type": "regex",
            "pattern": '(?i)ley(es)?(( y|,)? ([\d\.]+))+',
        },
    ] + 
    [
        {
            "label": "CODE_OR_LAW",
            "type": "regex",
            "pattern": f'{abbr}(\s?CABA)?',
        } for abbr in ABBRS
    ]
)

doc = nlp(examples)
doc = ruler(doc)
spacy.displacy.render(doc, "ent")
